In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.2 Safari/605.1.15"
        }

In [7]:
icd_chap_url=u'http://code.nhsa.gov.cn:8000/jbzd/public/toStdTcmTreeList.html?partcode=Z'
icd_chapter=requests.get(icd_chap_url,headers=headers)

In [8]:
icd_chapter=icd_chapter.json()

In [5]:
icd_chapter[0]['tcmId']

'36d1e171-7930-11e9-9611-8cec4bd010f3'

In [6]:
icd_chapter=pd.DataFrame(icd_chapter)
icd_chapter.to_csv('./CHS_TCM_SYMPTOM_CHAPTER.csv')

In [9]:
icd_detail_url_base=u'http://code.nhsa.gov.cn:8000/jbzd/public/dataTcmSerachDetail.html?tcmId='

In [10]:
icd_detail_url_full=icd_detail_url_base+icd_chapter[0]['tcmId']

In [11]:
icd_detail_url_full

'http://code.nhsa.gov.cn:8000/jbzd/public/dataTcmSerachDetail.html?tcmId=36d1e171-7930-11e9-9611-8cec4bd010f3'

In [12]:
icd_detail=requests.get(icd_detail_url_full,headers=headers)

In [13]:
if icd_detail.status_code == 200:
    info = {}
    soup = BeautifulSoup(icd_detail.text)
    icd_code_lst=[d.text.strip() for d in soup.find_all('a')]
    icd_name_lst=[d.text.strip() for d in soup.find_all('span')]


In [16]:
CHS_ICD10=[]
for i in range(len(icd_chapter)):
    icd_detail_url_full=icd_detail_url_base+icd_chapter[i]['tcmId']
    icd_detail=requests.get(icd_detail_url_full,headers=headers)
    if icd_detail.status_code == 200:
        info = {}
        soup = BeautifulSoup(icd_detail.text)
        icd_code_lst=[d.text.strip() for d in soup.find_all('a')]
        icd_name_lst=[d.text.strip() for d in soup.find_all('span')]
        df_icd=pd.DataFrame({'tcm_code':icd_code_lst,'tcm_name':icd_name_lst})
        CHS_ICD10.append(df_icd)
    else:
        print(icd_detail_url_full)

In [17]:
CHS_ICD10_Table=pd.concat(CHS_ICD10,axis=0,ignore_index=True)

In [18]:
CHS_ICD10_Table.head(20)

,tcm_code,tcm_name
0,ZBF,风证类
1,ZBF000,风邪证
2,ZBF001,风邪偏盛证
3,ZBF010,风邪袭表证
4,ZBF011,风邪外袭证
5,ZBF012,风邪外袭，经气痞塞证
6,ZBF013,风邪侵袭证
7,ZBF020,风邪袭络证
8,ZBF030,风邪阻络证
9,ZBF031,卫外失固，风邪中络证


In [19]:
len(CHS_ICD10_Table)

3284

In [20]:
CHS_ICD10_Table.to_csv('./LOOKUP_TABLES/CHS_TCM_SYMPTOM.csv',encoding='utf-8',index=False)